# DEEP LEANINNG MODEL - ConvNext-TINY

In [2]:
# === Cell 0: Imports, GPU check, path helper, seeds ===
import os, sys, time, json, math, platform, random, csv
from pathlib import PureWindowsPath
from collections import Counter

import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

def is_wsl():
    try:
        return ("microsoft" in platform.release().lower()) or ("wsl" in platform.version().lower())
    except Exception:
        return False

def win_to_wsl_path(win_path: str) -> str:
    if not is_wsl(): return win_path
    if ":" not in win_path: return win_path
    p = PureWindowsPath(win_path)
    drive = str(p.drive).replace(":", "").lower()
    tail = str(p).replace("\\", "/").split(":/")[-1]
    return f"/mnt/{drive}/{tail}"

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False  # keep False for speed
    torch.backends.cudnn.benchmark = True

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


Python: 3.12.3 (main, Aug 14 2025, 17:47:21) [GCC 13.3.0]
PyTorch: 2.8.0+cu128
CUDA available: True


device(type='cuda')

In [3]:
# === Cell 1: Config ===
# >>> UPDATE THIS PATH IF NEEDED <<<
BASE_DIR = r"C:\Users\sheno\OneDrive\CODCSD201F-006-SetupFile\Desktop\FINAL\dataset\OCT2017_70_15_15"
BASE_DIR = win_to_wsl_path(BASE_DIR)

OUT_DIR        = "convnext_tiny_run"
os.makedirs(OUT_DIR, exist_ok=True)

CLASSES        = ["CNV","DME","DRUSEN","NORMAL"]
NUM_CLASSES    = len(CLASSES)
IMG_SIZE       = 224         # ConvNeXt is trained on 224; you can try 128 for speed
BATCH_SIZE     = 32
NUM_WORKERS    = 4           # tune per your CPU
EPOCHS         = 15
LR             = 5e-4
WEIGHT_DECAY   = 0.05
LABEL_SMOOTH   = 0.1
PATIENCE       = 5           # early stopping

MODEL_NAME     = "convnext_tiny"
CHECKPOINT_BEST = os.path.join(OUT_DIR, f"{MODEL_NAME}_best.pt")
METRICS_CSV     = os.path.join(OUT_DIR, f"{MODEL_NAME}_metrics.csv")
CONF_JSON       = os.path.join(OUT_DIR, f"{MODEL_NAME}_confusion_matrix.json")

print("BASE_DIR:", BASE_DIR)


BASE_DIR: /mnt/c/Users/sheno/OneDrive/CODCSD201F-006-SetupFile/Desktop/FINAL/dataset/OCT2017_70_15_15


In [4]:
# === Cell 2: Ensure timm is available ===
try:
    import timm
except ImportError:
    !pip -q install timm
    import timm

# ImageNet normalization for ConvNeXt
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

train_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),     # OCT often grayscale -> 3-ch
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

val_tfms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(int(IMG_SIZE*1.14)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])


In [5]:
# === Cell 3: Datasets & Loaders ===
train_dir = os.path.join(BASE_DIR, "train")
val_dir   = os.path.join(BASE_DIR, "val")
test_dir  = os.path.join(BASE_DIR, "test")

for split in ("train","val","test"):
    sd = os.path.join(BASE_DIR, split)
    present = [c for c in CLASSES if os.path.isdir(os.path.join(sd, c))]
    print(f"[SCAN] {split}: {sd} -> classes: {present}")

ds_train = datasets.ImageFolder(train_dir, transform=train_tfms)
ds_val   = datasets.ImageFolder(val_dir,   transform=val_tfms)
ds_test  = datasets.ImageFolder(test_dir,  transform=val_tfms)

print("[INFO] Class->index mapping:", ds_train.class_to_idx)

def count_per_class(ds):
    counts = Counter([y for _, y in ds.samples])
    inv = {v:k for k,v in ds.class_to_idx.items()}
    return {inv[k]: counts.get(k,0) for k in sorted(counts)}

print("[INFO] Train counts:", count_per_class(ds_train))
print("[INFO] Val counts:",   count_per_class(ds_val))
print("[INFO] Test counts:",  count_per_class(ds_test))

loader_train = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
loader_val   = DataLoader(ds_val,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
loader_test  = DataLoader(ds_test,  batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)


[SCAN] train: /mnt/c/Users/sheno/OneDrive/CODCSD201F-006-SetupFile/Desktop/FINAL/dataset/OCT2017_70_15_15/train -> classes: ['CNV', 'DME', 'DRUSEN', 'NORMAL']
[SCAN] val: /mnt/c/Users/sheno/OneDrive/CODCSD201F-006-SetupFile/Desktop/FINAL/dataset/OCT2017_70_15_15/val -> classes: ['CNV', 'DME', 'DRUSEN', 'NORMAL']
[SCAN] test: /mnt/c/Users/sheno/OneDrive/CODCSD201F-006-SetupFile/Desktop/FINAL/dataset/OCT2017_70_15_15/test -> classes: ['CNV', 'DME', 'DRUSEN', 'NORMAL']
[INFO] Class->index mapping: {'CNV': 0, 'DME': 1, 'DRUSEN': 2, 'NORMAL': 3}
[INFO] Train counts: {'CNV': 26216, 'DME': 8116, 'DRUSEN': 6201, 'NORMAL': 18593}
[INFO] Val counts: {'CNV': 5618, 'DME': 1739, 'DRUSEN': 1329, 'NORMAL': 3984}
[INFO] Test counts: {'CNV': 5617, 'DME': 1739, 'DRUSEN': 1329, 'NORMAL': 3984}


In [6]:
# === Cell 4: Model, Loss, Optim, Sched ===
model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=NUM_CLASSES, drop_path_rate=0.1)
model = model.to(device)

# Label smoothing improves calibration
criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
print(model.__class__.__name__)


ConvNeXt


/tmp/ipykernel_85930/3839834633.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


In [7]:
# === Cell 5: Utilities ===
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

def top1_accuracy(logits, targets):
    with torch.no_grad():
        preds = logits.argmax(dim=1)
        return (preds == targets).float().mean().item()

def run_epoch(model, loader, train=True, max_norm=5.0):
    if train:
        model.train()
    else:
        model.eval()
    total_loss, total_acc, n = 0.0, 0.0, 0

    pbar = tqdm(loader, leave=False)
    for imgs, labels in pbar:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        with torch.amp.autocast(device_type='cuda', enabled=torch.cuda.is_available()):
            logits = model(imgs)
            loss = criterion(logits, labels)

        if train:
            optimizer.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            # gradient clipping
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            scaler.step(optimizer)
            scaler.update()

        acc = top1_accuracy(logits, labels)
        bs = imgs.size(0)
        total_loss += loss.item() * bs
        total_acc  += acc * bs
        n += bs
        pbar.set_description(f"{'Train' if train else 'Val'} loss {total_loss/n:.4f} | acc {total_acc/n:.4f}")
    return total_loss/n, total_acc/n


In [8]:
# === Cell 6: Training loop ===
best_val_acc = 0.0
pat_count = 0

# CSV header
if not os.path.isfile(METRICS_CSV):
    with open(METRICS_CSV, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["epoch","train_loss","train_acc","val_loss","val_acc","lr"])

t0 = time.time()
for epoch in range(1, EPOCHS+1):
    train_loss, train_acc = run_epoch(model, loader_train, train=True)
    val_loss,   val_acc   = run_epoch(model, loader_val,   train=False)

    # Step scheduler AFTER epoch
    scheduler.step()
    lr_now = scheduler.get_last_lr()[0]

    print(f"Epoch {epoch:02d}/{EPOCHS} | train_loss {train_loss:.4f} acc {train_acc:.4f} | "
          f"val_loss {val_loss:.4f} acc {val_acc:.4f} | lr {lr_now:.2e}")

    # log CSV
    with open(METRICS_CSV, "a", newline="") as f:
        w = csv.writer(f); w.writerow([epoch, f"{train_loss:.6f}", f"{train_acc:.6f}",
                                       f"{val_loss:.6f}", f"{val_acc:.6f}", f"{lr_now:.8f}"])

    # early stopping + checkpoint
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        pat_count = 0
        torch.save({"epoch": epoch,
                    "model_state": model.state_dict(),
                    "optimizer_state": optimizer.state_dict(),
                    "val_acc": best_val_acc,
                    "classes": CLASSES,
                    "img_size": IMG_SIZE}, CHECKPOINT_BEST)
        print(f"[SAVED] New best: val_acc={best_val_acc:.4f} -> {CHECKPOINT_BEST}")
    else:
        pat_count += 1
        if pat_count >= PATIENCE:
            print(f"[EARLY STOP] Patience {PATIENCE} reached at epoch {epoch}.")
            break

total_time = time.time() - t0
print(f"[TIMER] Training finished in {total_time/60:.1f} min")


Epoch 01/15 | train_loss 1.2778 acc 0.4388 | val_loss 1.2642 acc 0.4434 | lr 4.95e-04
[SAVED] New best: val_acc=0.4434 -> convnext_tiny_run/convnext_tiny_best.pt


Epoch 02/15 | train_loss 1.2656 acc 0.4433 | val_loss 1.2635 acc 0.4434 | lr 4.78e-04


Epoch 03/15 | train_loss 1.2654 acc 0.4433 | val_loss 1.2659 acc 0.4434 | lr 4.52e-04


Epoch 04/15 | train_loss 1.2649 acc 0.4434 | val_loss 1.2645 acc 0.4434 | lr 4.17e-04


Epoch 05/15 | train_loss 1.2647 acc 0.4431 | val_loss 1.2645 acc 0.4434 | lr 3.75e-04


Epoch 06/15 | train_loss 1.2646 acc 0.4431 | val_loss 1.2649 acc 0.4434 | lr 3.27e-04
[EARLY STOP] Patience 5 reached at epoch 6.
[TIMER] Training finished in 704.5 min


In [9]:
# === Cell 7: Evaluation on test set ===
# Load best
ckpt = torch.load(CHECKPOINT_BEST, map_location=device)
model.load_state_dict(ckpt["model_state"])
model.eval()

all_preds, all_labels = [], []
with torch.no_grad(), torch.amp.autocast(device_type='cuda', enabled=torch.cuda.is_available()):
    for imgs, labels in tqdm(loader_test, desc="Testing", leave=False):
        imgs = imgs.to(device, non_blocking=True)
        logits = model(imgs)
        preds = logits.argmax(dim=1).cpu().numpy()
        all_preds.append(preds)
        all_labels.append(labels.numpy())
all_preds  = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

acc = (all_preds == all_labels).mean()
print(f"[RESULT] Test Accuracy: {acc:.4f}")
print("\nClassification report:\n",
      classification_report(all_labels, all_preds, target_names=CLASSES, digits=4))

cm = confusion_matrix(all_labels, all_preds)
print("Confusion matrix:\n", cm)

# save confusion matrix JSON
with open(CONF_JSON, "w") as f:
    json.dump(cm.tolist(), f, indent=2)
print(f"[SAVED] {CONF_JSON}")


/home/sheno/tf/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


[RESULT] Test Accuracy: 0.4434

Classification report:
               precision    recall  f1-score   support

         CNV     0.4434    1.0000    0.6143      5617
         DME     0.0000    0.0000    0.0000      1739
      DRUSEN     0.0000    0.0000    0.0000      1329
      NORMAL     0.0000    0.0000    0.0000      3984

    accuracy                         0.4434     12669
   macro avg     0.1108    0.2500    0.1536     12669
weighted avg     0.1966    0.4434    0.2724     12669

Confusion matrix:
 [[5617    0    0    0]
 [1739    0    0    0]
 [1329    0    0    0]
 [3984    0    0    0]]
[SAVED] convnext_tiny_run/convnext_tiny_confusion_matrix.json


/home/sheno/tf/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/sheno/tf/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [10]:
# === Cell 8: Inference helper ===
from PIL import Image

infer_tfms = val_tfms  # same as validation

idx_to_class = {v:k for k,v in ds_train.class_to_idx.items()}

def predict_image(path: str):
    path = win_to_wsl_path(path)
    img = Image.open(path).convert("L")  # grayscale load
    img = transforms.functional.to_pil_image(np.array(img))
    img = infer_tfms(img).unsqueeze(0).to(device)

    with torch.no_grad(), torch.amp.autocast(device_type='cuda', enabled=torch.cuda.is_available()):
        logits = model(img)
        prob = torch.softmax(logits, dim=1)[0].cpu().numpy()
        pred_idx = int(np.argmax(prob))
    return idx_to_class[pred_idx], {idx_to_class[i]: float(prob[i]) for i in range(len(prob))}

# Example:
# predict_image(r"C:\path\to\one\OCT\image.jpeg")


In [11]:
# === Save Final Model ===
FINAL_MODEL_PATH = os.path.join(OUT_DIR, f"{MODEL_NAME}_final.pt")

torch.save({
    "model_state": model.state_dict(),
    "optimizer_state": optimizer.state_dict(),
    "epoch": epoch,
    "classes": CLASSES,
    "img_size": IMG_SIZE
}, FINAL_MODEL_PATH)

print(f"[SAVED] Final model -> {FINAL_MODEL_PATH}")


[SAVED] Final model -> convnext_tiny_run/convnext_tiny_final.pt


In [12]:
# === Load Final Model ===
ckpt = torch.load(FINAL_MODEL_PATH, map_location=device)

model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=len(ckpt["classes"]))
model.load_state_dict(ckpt["model_state"])
model = model.to(device)
model.eval()

print("Model loaded and ready for inference ✅")


Model loaded and ready for inference ✅
